In [1]:
from skimage.color import rgb2gray, rgb2hsv
from matplotlib.pyplot import bar
from skimage.exposure import histogram
from skimage.util import random_noise
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.filters import median
from skimage.filters import gaussian
from skimage.color import rgb2gray,rgb2hsv
# import the opencv library
import cv2
#new
from scipy import ndimage
from datetime import datetime
import numpy as np
import math

# Show the figures / plots inside the notebook
%matplotlib inline


In [2]:

def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


In [3]:

import cv2 
import numpy as np
import math
import skimage.io as io

#set initial values
flag = {"first": False, "operator": False, "second": False}
count_same_frames = 0
first, operator, second = "", "", ""
pred_text = ""
calc_text = ""
info = "Enter first number"        

# define a video capture object
vid = cv2.VideoCapture(0)

while (True):

#    # Capture the video frame
#     # by frame
#     ret, frame = vid.read()
#     dim = (1000, 800)
#     frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
#     # Display the resulting frame
#     frame = cv2.flip(frame, 1)
   
#     print(frame.shape)
#     border=frame[500:990, 400:790]
#      #blur the image
#     frame = cv2.GaussianBlur(frame,(5,5),100) 
#     frame = cv2.medianBlur(frame,7)
#     cv2.rectangle(frame,(500,400),(990,790),(0, 0, 0),2)
#     hsv = cv2.cvtColor(border, cv2.COLOR_BGR2HSV)
#     # define range of skin color in HSV
#     # lower_skin = np.array([0,20,70], dtype=np.uint8)
#     # upper_skin = np.array([20,255,255], dtype=np.uint8)
#     lower_skin = np.array([0,15,0], dtype=np.uint8)
#     upper_skin = np.array([17,170,255], dtype=np.uint8)
        
#      #extract skin colur imagw  
#     mask = cv2.inRange(hsv, lower_skin, upper_skin)
#     kernel = np.ones((3,3),np.uint8)
   
        
   
        
#     # #extrapolate the hand to fill dark spots within
#     mask = cv2.dilate(mask,kernel,iterations = 5)
#     # mask = cv2.erode(mask,kernel,iterations = 1)   
# cv2.imshow('frame',frame)
    ret, frame = vid.read()
    dim = (1000, 800)
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    # Display the resulting frame
    frame = cv2.flip(frame, 1)
    # Get hand data from the rectangle sub window   
    cv2.rectangle(frame,(500,400),(990,790),(0, 0, 0),2)
    crop_image = frame[500:990, 400:790]
    
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(crop_image, (3,3), 0)
    
    # Change color-space from BGR -> HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    
    # Create a binary image with where white will be skin colors and rest is black
    mask2 = cv2.inRange(hsv, np.array([0,0,0]), np.array([20,255,255]))
    
    # Kernel for morphological transformation    
    kernel = np.ones((5,5))
    
    # Apply morphological transformations to filter out the background noise
    dilation = cv2.dilate(mask2, kernel, iterations = 1)
    erosion = cv2.erode(dilation, kernel, iterations = 1)    
       
    # Apply Gaussian Blur and Threshold
    mask = cv2.GaussianBlur(erosion, (3,3), 0)
    
    
    #detection
    #find contours

# binaryHand = 255 - binaryHand;
# contours,hierarchy= cv2.findContours(binaryHand,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# # print(contours)

    ret, thresh = cv2.threshold(mask, 127, 255,0)
    contours, hierarchy = cv2.findContours(thresh,2,1)
    #new added 
    old_pred_text = pred_text
    if len(contours)!=0:
        #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        # print(max(contours, key = lambda x: cv2.contourArea(x)))


        #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)
        #approx=cnt
        # cv2.imshow('binaryHand', binaryHand);
        # cv2.waitKey(0)
        # io.imshow(binaryHand)
        # io.show()

        #make convex hull around hand
        hull = cv2.convexHull(cnt)
        print(cnt)
        #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)

    #find the percentage of area not covered by hand in convex hull
        if areacnt!=0:
            arearatio=((areahull-areacnt)/areacnt)*100
        else :
            arearatio=0    
        #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        if len(hull)!=0:
            defects = cv2.convexityDefects(approx, hull)
            # print(defects.shape)
            l=0   
        #     if True:
            if len(defects)!=0:

                #code for finding no. of defects due to fingers
                for i in range(defects.shape[0]):
                    s,e,f,d = defects[i,0]
                    start = tuple(approx[s][0])
                    end = tuple(approx[e][0])
                    far = tuple(approx[f][0])
                    pt= (100,180)


                    # find length of all sides of triangle
                    a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    s = (a+b+c)/2
                    area_triangle  = math.sqrt(s*(s-a)*(s-b)*(s-c))

                    #distance between point and convex hull
                    d=(2*area_triangle)/a

                    # apply cosine rule here
                    angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
                    print(angle)        

                    # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
                    if angle <= 90 and d>30:
                        l += 1
                        #cv2.circle(img, far, 7, [255,0,0], -1)

                    #draw lines around hand
                    cv2.line(crop_image,start, end, [0,255,0], 2)
                    #if(i == 0):
                    cv2.circle(crop_image, start, 4, [255,0,0], -1)
                    cv2.circle(crop_image, end, 4, [0,0,255], -1)#[0,0,255]:red
                    cv2.circle(crop_image, far, 3, [0,255,0], -1)
            #cv2.imshow('final_img',img)
            #cv2.waitKey(0)
            #time.sleep(6)
            #if(i == 0):
                #cv2.circle(img, start, 4, [0,0,255], -1)
                #cv2.circle(img, end, 4, [0,0,0], -1)    


                    l+=1
            # cv2.imshow('final_img',img)
            # cv2.waitKey(0)
            font = cv2.FONT_HERSHEY_SIMPLEX
            # frame=img
            pred_text = l
            if old_pred_text == pred_text:
                    count_same_frames += 1
            else:
                    count_same_frames = 0
	        
            if count_same_frames > 5 :
                # count_same_frames = 0
                # first, second, operator, pred_text, calc_text = '', '', '', '', ''
				# flag['first'], flag['operator'], flag['second'], flag['clear'] = False, False, False, False
				# info = "Enter first number"
                if  flag['first'] == False and flag['operator'] == False and flag['second'] == False :
                     count_same_frames = 0
                     first, second, operator, pred_text, calc_text = pred_text, '', '', '', ''
                     info = "Enter the Sign"
                elif flag['first'] == True and flag['operator'] == False and flag['second'] == False :
                     count_same_frames = 0
                     #operator will be changed later
                     second, operator, pred_text, calc_text = '', pred_text, '', ''
                     info = "Enter the second operand"
                elif flag['first'] == True and flag['operator'] == True and flag['second'] == False :
                     count_same_frames = 0
                     second, pred_text, calc_text = pred_text,'', ''
                     info = "all is well !"     

            if l==1:
            #     if areacnt<2000:
            #         cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            #     else:
            #     if arearatio<12:
            #         cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            #     elif arearatio<17.5:
            #         cv2.putText(frame,'Best of luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            #     else:
                cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==2:
                cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==3:

            #     if arearatio<27:
                cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            #     else:
            #         cv2.putText(frame,'ok',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==4:
                cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==5:
                cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==6:
                cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            else :
                cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
           


    cv2.imshow('frame',frame)        
    cv2.imshow('mask',mask)  

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
   
    # cv2.imshow('frame',frame)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    # wait variable is to calculate waiting time
    # key = cv2.waitKey(100)
    # wait = wait+100
    
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    # if key == ord('q'):
    #     break
    # if wait == 5000:
    #     arr[i]=mask
    #     i = i+1
        # cv2.imshow('mask',mask)

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()


[[[25  0]]

 [[25  1]]

 [[25  2]]

 ...

 [[28  0]]

 [[27  0]]

 [[26  0]]]
95.21650381930534
143.1955248402326
83.0857846766515
51.94842856103364
[[[26  0]]

 [[26  1]]

 [[26  2]]

 ...

 [[29  0]]

 [[28  0]]

 [[27  0]]]
105.39847119060046
173.36050385017043
80.86617660662994
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
113.68320828706406
81.78403450739698
121.71584606639078
171.8747379936567
79.66844569517016
87.16080969488306
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
113.68320828706406
81.78403450739698
121.71584606639078
171.8747379936567
79.66844569517016
87.16080969488306
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
113.68320828706406
81.78403450739698
121.71584606639078
171.8747379936567
79.66844569517016
87.16080969488306
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
109.57573788294874
82.02996

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'


: 